In [1]:
!pip install openai
!pip install langchain
!pip install langchain_community
!pip install langchain_openai
!pip install tiktoken
!pip install pydantic
!pip install python-dotenv

When we want to convert openai functions to pydantic objects (for a more structured approach for defining tools for LLMs) we can leverage the following ideas:

In [2]:
from pydantic import BaseModel, Field
from langchain.utils.openai_functions import convert_pydantic_to_openai_function


class WeatherSearch(BaseModel):
    """Call this function to get the weather of a specific airport."""
    airport_code: str = Field(description="The airport code of the airport you want to know the weather of.")
    

openai_function_weather_search = convert_pydantic_to_openai_function(WeatherSearch)

/var/folders/5l/y8s3fc655417629rqwgxkhx80000gn/T/ipykernel_43343/1406677583.py:10: LangChainDeprecationWarning: The function `convert_pydantic_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 1.0. Use :meth:`~langchain_core.utils.function_calling.convert_to_openai_function()` instead.
  openai_function_weather_search = convert_pydantic_to_openai_function(WeatherSearch)


SO the airport_code is something that the LLM will figure out on how to input to the function, therefore making the call with the correct input.

In [3]:
from langchain.chat_models import ChatOpenAI

llm_chat = ChatOpenAI()

llm_chat.invoke("What's the weather like in Lisbon today?",functions=[openai_function_weather_search])

/var/folders/5l/y8s3fc655417629rqwgxkhx80000gn/T/ipykernel_43343/1600133725.py:3: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm_chat = ChatOpenAI()


AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"LIS"}', 'name': 'WeatherSearch'}}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 76, 'total_tokens': 94, 'completion_tokens_details': {'audio_tokens': 0, 'reasoning_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-4c9ff3f5-9ec1-4a4b-b5ae-1ed3846106ee-0')

We can see from below that the LLM figured out that the airport_code for Lisbon is `LIS`.

You can also bind the function to the model to avoid having to pass the `functions` keyword argument everytime. 

In [4]:
model_with_function = llm_chat.bind(functions=[openai_function_weather_search])
model_with_function.invoke("What's the weather like in Lisbon today?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"LIS"}', 'name': 'WeatherSearch'}}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 76, 'total_tokens': 94, 'completion_tokens_details': {'audio_tokens': 0, 'reasoning_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-f76fab10-6afa-4ecf-9077-29b0a3124900-0')

YOu can also force the model to use the function by binding it with it and adding the `function_call` keyword argument to a dictionary containing the key `name` and the name of that function `WeatherSearch`:

In [5]:
model_with_forced_function = llm_chat.bind(functions=[openai_function_weather_search],function_call={"name": "WeatherSearch"})

Now, let's connect to the stuff we already know.

In [6]:
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that tells the weather"),
    ("user", "{input}")
])

llm_chat = ChatOpenAI(temperature=0)

llm_chat_with_forced_function = llm_chat.bind(functions=[openai_function_weather_search], function_call={"name": "WeatherSearch"})

chain = prompt | llm_chat_with_forced_function

chain.invoke({"input": "What's the weather like in Lisbon today?"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"LIS"}', 'name': 'WeatherSearch'}}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 96, 'total_tokens': 104, 'completion_tokens_details': {'audio_tokens': 0, 'reasoning_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-f95d2bf4-088c-4283-9418-afe9ab2b82d7-0')

For multiple functions:

In [7]:
from typing import List

class SearchExamples(BaseModel):
    """Call this function to search for examples of a concept."""
    concept: str = Field(description="The concept you want to search for.")
    
class AnalogySearch(BaseModel):
    """Call this function to search for analogies of a concept."""
    concept: str = Field(description="The concept you want to search for.")


openai_function_analogy_search = convert_pydantic_to_openai_function(AnalogySearch)
openai_function_example_search = convert_pydantic_to_openai_function(SearchExamples) 

In [8]:
functions = [
    openai_function_analogy_search,
    openai_function_example_search
]

In [9]:
llm_chat_with_functions = llm_chat.bind(functions=functions)

In [10]:
# Here the model calls the example function
llm_chat_with_functions.invoke("List examples for what and SDK is.")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"concept":"SDK"}', 'name': 'SearchExamples'}}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 105, 'total_tokens': 120, 'completion_tokens_details': {'audio_tokens': 0, 'reasoning_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-63796f7d-1ffe-4fee-94d6-35ac39f6c532-0')

In [11]:
llm_chat_with_functions.invoke("List analogies for what and SDK is.")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"concept":"SDK"}', 'name': 'AnalogySearch'}}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 106, 'total_tokens': 122, 'completion_tokens_details': {'audio_tokens': 0, 'reasoning_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-6d01fde9-8b0e-427d-986a-c2894203dc83-0')